This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [1]:
!kaggle competitions download -c kitchenware-classification
!mkdir data
!unzip kitchenware-classification.zip -d data > /dev/null
!rm kitchenware-classification.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file data already exists.
The system cannot find the path specified.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Now let's train a baseline model

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [4]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 8
WORKERS = 4
EPOCHS = 10

BASE_PATH='./data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [5]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [6]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [7]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [9]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [10]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

callbacks = [earlystopping]

In [ ]:
EXP_NAME = 'model_search_transformers_large'
mlflow.set_experiment(EXP_NAME)
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.tensorflow.autolog()

In [11]:
from keras_cv_attention_models import beit, efficientnet, coatnet, davit, convnext, hornet, swin_transformer_v2, maxvit

In [ ]:
base_model = beit.BeitLargePatch16( #BeitBasePatch16(
    pretrained='imagenet21k-ft1k',  #weights='imagenet',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    num_classes=len(classes)
)
base_model.trainable = False

>>>> Load pretrained from: C:\Users\andre\.keras\models\beit_large_patch16_224_imagenet21k-ft1k.h5


In [ ]:
base_model = tf.keras.Model(inputs=base_model.layers[1].input, outputs= base_model.layers[-3].output)

In [ ]:
inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = tf.keras.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, 'gelu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, 'softmax')
],
name = 'efficientnet2_xl')

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)

Epoch 1/2
556/556 [==============================] - 140s 217ms/step - loss: 0.2007 - accuracy: 0.9445 - val_loss: 0.0806 - val_accuracy: 0.9793
Epoch 2/2
423/556 [=====================>........] - ETA: 21s - loss: 0.1085 - accuracy: 0.9642

Now let's use this model to predict the labels for test data

In [ ]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,data/images/0678.jpg
1,3962,data/images/3962.jpg
2,9271,data/images/9271.jpg
3,5133,data/images/5133.jpg
4,8842,data/images/8842.jpg


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [ ]:
y_pred = model.predict(test_generator)

InvalidArgumentError: Graph execution error:

Detected at node 'beit_transformer/flatten/Reshape' defined at (most recent call last):
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\tornado\platform\asynci

In [ ]:
predictions = classes[y_pred.argmax(axis=1)]

Finally, we need to prepare the submission

In [ ]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('data/images/'):-4]
del df_submission['filename']

In [ ]:
df_submission[['Id', 'label']].to_csv('submission.csv', index=False)

In [ ]:
# !kaggle competitions submit kitchenware-classification -f submission.csv -m 'validation: 0.8570'